<a href="https://colab.research.google.com/github/fallensnake/Kalshi-Bots/blob/main/AnalyzePortfolio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import time
import base64
import requests
import datetime
import env
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import serialization
from google.colab import userdata

ModuleNotFoundError: No module named 'env'

In [ ]:
# --- IMPORT FROM YOUR ENV FILE ---

def get_private_key():
    """Loads the private key directly from the imported env module."""
    return serialization.load_pem_private_key(
        env.KALSHI_PRIVATE_KEY_PATH.encode(),
        password=None
    )

def sign_request(method, path):
    """Generates the RSA signature using the key from env.py."""
    timestamp = str(int(time.time() * 1000))
    msg_string = timestamp + method + path

    private_key = get_private_key()
    signature = private_key.sign(
        msg_string.encode('utf-8'),
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )
    return timestamp, base64.b64encode(signature).decode('utf-8')

def make_authenticated_request(method, endpoint, params=None):
    """Sends signed requests using credentials from env.py."""
    path = '/trade-api/v2' + endpoint
    timestamp, signature = sign_request(method, path)

    headers = {
        "KALSHI-ACCESS-KEY": env.KALSHI_KEY_ID,  # Accessing ID from env.py
        "KALSHI-ACCESS-SIGNATURE": signature,
        "KALSHI-ACCESS-TIMESTAMP": timestamp,
        "Content-Type": "application/json"
    }

    url = f"https://api.elections.kalshi.com/trade-api/v2{endpoint}"
    response = requests.request(method, url, headers=headers, params=params)
    return response.json() if response.status_code == 200 else None

# ... rest of the portfolio logic follows ...

# --- MAIN PORTFOLIO FUNCTIONS ---

def get_portfolio_summary():
    """Fetches balance and active positions."""
    print("💼 Accessing Portfolio...")

    # 1. Get Balance
    balance_data = make_authenticated_request("GET", "/portfolio/balance")
    if not balance_data:
        return

    balance_cents = balance_data.get('balance', 0)
    cash = balance_cents / 100.0
    print(f"\n💰 Available Cash: ${cash:,.2f}")

    # 2. Get Active Positions
    # status='settled' gets old ones, default (open) gets current ones
    positions_data = make_authenticated_request("GET", "/portfolio/positions")

    positions = positions_data.get('market_positions', [])
    if not positions:
        print("📭 No active positions found.")
        return

    print(f"\n📊 Current Holdings ({len(positions)} markets):")
    print("-" * 60)
    print(f"{'Market Ticker':<25} | {'Side':<4} | {'Count':<5} | {'Avg Price':<10} | {'Current':<10} | {'P&L'}")
    print("-" * 60)

    for p in positions:
        ticker = p.get('ticker')
        # Negative count = NO, Positive count = YES
        raw_count = p.get('position', 0)
        side = "YES" if raw_count > 0 else "NO"
        count = abs(raw_count)

        # Calculate Average Entry Price
        # Cost basis is negative for buys, so we flip sign
        cost_basis = p.get('fees_paid', 0) # simplified, usually need 'cost_basis' from fills
        # Kalshi 'market_positions' object structure can be tricky, using simplified math here
        # For precise entry price, we often need to divide total_cost / count

        # 3. Get Current Market Price for this ticker (Public Endpoint)
        market_url = f"https://api.elections.kalshi.com/trade-api/v2/markets/{ticker}"
        try:
            m_data = requests.get(market_url).json()['market']
            curr_price = m_data.get('yes_bid') if side == "YES" else (100 - m_data.get('yes_ask', 0))

            # Simple Entry Price Estimate (Avg Price)
            # This is an approximation. Real P&L requires tracking specific fills.
            # We assume fees_paid roughly tracks cost for this display.
            # In a real app, you'd use the 'fills' endpoint for exact entry price.

            # Formatting
            print(f"{ticker:<25} | {side:<4} | {count:<5} | {'--':<10} | {curr_price}¢      | --")

        except Exception as e:
            print(f"{ticker:<25} | {side:<4} | {count:<5} | (Error fetching price)")

    print("-" * 60)